In [1]:
corpus = [
    "SELECT * FROM customers WHERE active = 1;",
    "INSERT INTO products (name, price) VALUES ('Laptop', 1200);",
    "UPDATE employees SET salary = 5000 WHERE department = 'HR';",
    "DELETE FROM orders WHERE order_date < '2023-01-01';",
    "CREATE TABLE users (id INT PRIMARY KEY, username VARCHAR(50));",
    "ALTER TABLE invoices ADD COLUMN total DECIMAL(10, 2);",
    "DROP TABLE IF EXISTS temp_data;",
    "SELECT name, COUNT(*) AS total FROM sales GROUP BY name;",
    "JOIN addresses ON customers.id = addresses.customer_id;",
    "ORDER BY created_at DESC LIMIT 10;",
    "WHERE status = 'pending' AND priority > 5;",
    "HAVING SUM(amount) > 1000;",
    "GRANT SELECT, INSERT ON database.* TO 'user'@'localhost';",
    "REVOKE ALL PRIVILEGES ON employees FROM 'intern'@'%';",
    "BEGIN TRANSACTION; COMMIT;",
    "ROLLBACK TO SAVEPOINT before_update;",
    "EXPLAIN SELECT * FROM logs WHERE type = 'error';",
    "INDEX idx_name ON employees (last_name);",
    "PRIMARY KEY (order_id, product_id);",
    "FOREIGN KEY (customer_id) REFERENCES customers(id);"
]

# Pré-processamento

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter

# Tokenização e vocabulário
tokens = [word.lower() for sentence in corpus for word in sentence.split()]
vocab = Counter(tokens)
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_size = len(vocab)

# Mapeamento palavra para índice
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

Nesta etapa, vamos criar o modelo de linguagem que será treinado para prever a próxima palavra com base no contexto das palavras anteriores. O modelo será uma rede neural baseada em embeddings e LSTMs, que são adequados para lidar com sequências de texto.

---

### **Estrutura do Modelo**

O modelo será composto por:

1. **Camada de Embedding**  
   - Converte os índices das palavras em vetores densos de tamanho fixo (representação vetorial das palavras).  
   - Tamanho do embedding (`embedding_dim`) é um hiperparâmetro.

2. **Camada LSTM**  
   - Processa as sequências de embeddings e mantém informações sobre o contexto anterior.  
   - Tamanho do estado oculto (`hidden_dim`) é outro hiperparâmetro.

3. **Camada Linear (Fully Connected)**  
   - Projeta a saída da LSTM para o espaço do vocabulário, gerando scores para cada palavra no vocabulário.

4. **Softmax (aplicado durante inferência)**  
   - Converte os scores para probabilidades de cada palavra no vocabulário.

In [3]:
class SQLLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

Na sequência, vamos criar um **Dataset** e um **DataLoader** para alimentar o modelo com dados durante o treinamento. O Dataset será responsável por preparar as sequências de entrada e saída a partir do corpus, enquanto o DataLoader organizará os dados em lotes (`batches`) para treinamento eficiente.

---

### **Estrutura do Dataset**

O Dataset utilizará o corpus pré-processado e fará o seguinte:

1. **Divisão em Sequências**  
   - Cada sentença será dividida em sequências de comprimento fixo (`seq_length`).  
   - Por exemplo, dado o texto: `"SELECT * FROM customers"`, com `seq_length=3`:
     - Entrada (`x`): `"SELECT * FROM"`
     - Saída (`y`): `"* FROM customers"`

2. **Mapeamento para Índices**  
   - Cada palavra será convertida para seu índice correspondente no vocabulário (`word_to_idx`).

3. **Par Entrada-Saída**  
   - Para cada sequência de entrada (`x`), haverá uma sequência de saída (`y`), que corresponde às próximas palavras.

---


In [5]:
class SQLDataset(Dataset):
    def __init__(self, corpus, seq_length=3):
        self.seq_length = seq_length
        self.data = []

        for sentence in corpus:
            tokens = sentence.lower().split()  # Dividir a frase em palavras
            indices = [word_to_idx[word] for word in tokens]  # Mapear palavras para índices
            for i in range(len(indices) - self.seq_length):
                self.data.append((
                    torch.tensor(indices[i:i+self.seq_length]),
                    torch.tensor(indices[i+1:i+1+self.seq_length])
                ))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = SQLDataset(corpus, seq_length=3)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

Agora que temos o **modelo**, o **Dataset** e o **DataLoader**, o próximo passo é implementar o loop de treinamento. O objetivo do treinamento é ajustar os pesos do modelo para minimizar a perda (loss), que mede a diferença entre as previsões do modelo e as saídas reais.

---

### **Etapas do Treinamento**

1. **Definir Função de Perda e Otimizador**  
   - A função de perda será a `CrossEntropyLoss`, que é adequada para problemas de classificação, como prever a próxima palavra.  
   - O otimizador será o `Adam`, que é eficiente para redes neurais profundas.

2. **Loop de Treinamento**  
   - Para cada época:
     - Iterar sobre os lotes do DataLoader.
     - Passar o lote pelo modelo para obter as previsões.
     - Calcular a perda comparando as previsões com as saídas reais.
     - Atualizar os pesos do modelo usando o otimizador.

3. **Monitorar o Progresso**  
   - Exibir a perda a cada época para acompanhar o aprendizado do modelo.



In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SQLLM(vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 200
for epoch in range(epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        output, _ = model(inputs)
        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()


    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: 4.5007
Epoch 2, Loss: 4.0063
Epoch 3, Loss: 3.4762
Epoch 4, Loss: 2.4731
Epoch 5, Loss: 1.1435
Epoch 6, Loss: 1.0793
Epoch 7, Loss: 0.7201
Epoch 8, Loss: 0.8749
Epoch 9, Loss: 0.6145
Epoch 10, Loss: 0.4538
Epoch 11, Loss: 0.3816
Epoch 12, Loss: 0.7913
Epoch 13, Loss: 0.1348
Epoch 14, Loss: 0.1684
Epoch 15, Loss: 0.1087
Epoch 16, Loss: 0.1073
Epoch 17, Loss: 0.1309
Epoch 18, Loss: 0.5731
Epoch 19, Loss: 0.1174
Epoch 20, Loss: 0.0730
Epoch 21, Loss: 0.7512
Epoch 22, Loss: 0.0378
Epoch 23, Loss: 0.0483
Epoch 24, Loss: 0.0260
Epoch 25, Loss: 0.0603
Epoch 26, Loss: 0.0285
Epoch 27, Loss: 0.0354
Epoch 28, Loss: 0.0248
Epoch 29, Loss: 0.0161
Epoch 30, Loss: 0.5270
Epoch 31, Loss: 0.0322
Epoch 32, Loss: 0.0152
Epoch 33, Loss: 0.5958
Epoch 34, Loss: 0.7004
Epoch 35, Loss: 0.0132
Epoch 36, Loss: 0.3817
Epoch 37, Loss: 0.0065
Epoch 38, Loss: 0.0267
Epoch 39, Loss: 0.0159
Epoch 40, Loss: 0.0100
Epoch 41, Loss: 0.0073
Epoch 42, Loss: 0.0049
Epoch 43, Loss: 0.5871
Epoch 44, Loss: 0.58

# Função de completar texto

In [8]:
def complete_text(seed_text, num_words=5, temperature=0.7):
    model.eval()
    words = seed_text.lower().split()

    with torch.no_grad():
        for _ in range(num_words):
            inputs = torch.tensor(
                [word_to_idx[word] for word in words[-3:]]  # Usa contexto de 3 palavras
            ).unsqueeze(0).to(device)

            output, _ = model(inputs)
            probabilities = torch.softmax(output[0, -1] / temperature, dim=0)
            next_idx = torch.multinomial(probabilities, 1).item()

            words.append(idx_to_word[next_idx])

    return " ".join(words).capitalize()

In [9]:
# Exemplos de prompts
print(complete_text("SELECT * FROM", num_words=5))
print(complete_text("ADD COLUMN total", num_words=3))
print(complete_text("ROLLBACK TO SAVEPOINT", num_words=4))

Select * from logs where type = 'error';
Add column total decimal(10, 2); group
Rollback to savepoint before_update; where status =
